In [33]:
import StringIO
import ipywidgets as widgets
import numpy as np
import pandas as pd
from IPython.display import display
from traitlets import Unicode
from traitlets import default
from traitlets import List

In [52]:
class SciSheetTableWidget(widgets.DOMWidget):

    _view_name = Unicode('SciSheetTableView').tag(sync=True)
    _model_name = Unicode('SciSheetTableModel').tag(sync=True)
    _view_module = Unicode('jupyter_scisheets_widget').tag(sync=True)
    _model_module = Unicode('jupyter_scisheets_widget').tag(sync=True)
    _model_data = List([]).tag(sync=True)
    _model_data_2 = Unicode().tag(sync=True)

    @default('layout')
    def _default_layout(self):
        return widgets.Layout(height='400px', align_self='stretch')

In [56]:
tbwidge = SciSheetTableWidget()

var json = this.model.get('value');
        
// Parse it into data (list of lists)
var data = JSON.parse(json);
        

TraitError: The '_model_data_2' trait of a SciSheetTableWidget instance must be a unicode string, but a value of      0    1    2    3    4
0  898  610  409  557  126
1  603  175  869  412  856
2  220  526  412  562  140 <class 'pandas.core.frame.DataFrame'> was specified.

In [44]:
%%javascript
;(function(require){
  "use strict";
  var window = this;
  require([
    "@jupyter-widgets/base",
    "underscore",
    "handsontable/dist/handsontable.full.css",
    "http://handsontable.com/dist/jquery.handsontable.full.js"
  ], function(IPython, manager, widget, _, $){
    // Just once, ensure that the scroll event is called on the window for HoT
    var $win = $(window);
    IPython.notebook.element.on("scroll", function(){ $win.scroll(); });

    // Define the HandsonTableView
    var SciSheetTableView = widget.DOMWidgetView.extend({

      render: function(){
        // CREATION OF THE WIDGET IN THE NOTEBOOK.
        // Add a <div> in the widget area.
        this.$table = $('<div />').appendTo(this.$el);
        
        var view = this;

        // Create the Handsontable table.
        this.$table.handsontable({
          // when working in HoT, don't listen for command mode keys
          afterSelection: function(){ IPython.keyboard_manager.disable(); },
          afterDeselect: function(){ IPython.keyboard_manager.enable(); },
          // the data changed. `this` is the HoT instance
          afterChange: function(changes, source){
            // don't update if we did the changing!
            if(source === "loadData"){ return; }
            view.handle_table_change(this.getData());
          }
        });
      },

      update: function() {
        // PYTHON --> JS UPDATE.

        // Get the model's value (JSON)
        var json = this.model.get('value');
        
        // Parse it into data (list of lists)
        var data = JSON.parse(json);
        
        // Give it to the Handsontable widget.
        this.$table.handsontable({data: data});
        
        // Don't touch this...
        return SciSheetTableView.__super__.update.apply(this);
      },

      handle_table_change: function(data) {
        // JS --> PYTHON UPDATE.
        // Update the model with the JSON string.
        this.model.set('value', JSON.stringify(data));

        // Don't touch this...
        this.touch();
      }
    });
    // Register the HandsonTableView with the widget manager.
    manager.WidgetManager.register_widget_view('SciSheetTableView', SciSheetTableView);
  });
}).call(this, require);

<IPython.core.display.Javascript object>

In [ ]:
class SciSheetWidgetDataFrame(object):
    def __init__(self, df):
        self._df = df
        self._widget = SciSheetTableWidget()
        self._widget.observe(self._on_data_changed, '_model_data')
        self._widget.unobserve(self._on_displayed)
        
    def _on_displayed(self, e):
        # DataFrame ==> Widget (upon initialization only)
        json = self._df.to_json(orient='values')
        self._widget._model_data = json
        
    def _on_data_changed(self, e, val):
        # Widget ==> DataFrame (called every time the user
        # changes a value in the graphical widget)
        buf = StringIO.StringIO(val)
        self._df = pd.read_json(buf, orient='values')
        
    def to_dataframe(self):
        return self._df
        
    def show(self):
        display(self._widget)


In [12]:
from IPython.display import display

In [2]:
from jupyter_scisheets_widget import scisheets_widget

In [49]:
tb = scisheets_widget.SciSheetTable()

In [51]:
tb._model_data = df

TraitError: The '_model_data' trait of a SciSheetTable instance must be a list, but a value of class 'pandas.core.frame.DataFrame' (i.e.      0    1    2    3    4
0  898  610  409  557  126
1  603  175  869  412  856
2  220  526  412  562  140) was specified.

In [3]:
data = np.random.randint(size=(3, 5), low=100, high=900)
df = pd.DataFrame(data)
df

,0,1,2,3,4
0,898,610,409,557,126
1,603,175,869,412,856
2,220,526,412,562,140


In [ ]:
def _on_displayed(test_df, test_widget, e):
    # DataFrame ==> Widget (upon initialization only)
    json = test_df.to_json(orient='values')
    test_widget.value = json
    
    return test_widget

def _on_data_changed(test_df, e, val):
    # Widget ==> DataFrame (called every time the user
    # changes a value in the graphical widget)
    buf = io.StringIO(val)
    test_df = pd.read_json(buf, orient='values')
    
    return test_df


test_df = df
test_widget = scisheets_widget.SciSheetTable()
test_widget.on_trait_change(self._on_data_changed, 'value')
self._widget.on_displayed(self._on_displayed)

In [7]:
test_df = df
test_widget = scisheets_widget.SciSheetTable()


In [9]:
test_widget.unobserve()

TypeError: on_displayed() takes at least 2 arguments (1 given)

In [45]:
class HandsonDataFrame(object):
    def __init__(self, df):
        self._df = df
        self._widget = scisheets_widget.SciSheetTable()
        self._widget.observe(self._on_value_change, '_model_data')
        self._widget.unobserve(self._on_displayed)
        
        
    def _on_value_change(change):
        print(change['new'])
  
    def _on_displayed(self, e):
        # DataFrame ==> Widget (upon initialization only)
        json = self._df.to_json(orient='values')
        self._widget._model_data = json
        
    def _on_data_changed(self, e, val):
        # Widget ==> DataFrame (called every time the user
        # changes a value in the graphical widget)
        buf = io.StringIO(val)
        self._df = pd.read_json(buf, orient='values')
        
    def to_dataframe(self):
        return self._df
        
    def show(self):
        display(self._widget)

In [25]:
import ipywidgets as widgets

In [46]:
ht = HandsonDataFrame(df)
#ht.show()

In [47]:
ht.show()

A Jupyter Widget

In [48]:
ht.to_dataframe()

,0,1,2,3,4
0,898,610,409,557,126
1,603,175,869,412,856
2,220,526,412,562,140


In [9]:
%%javascript

var handsontable = require(["handsontable"]);

<IPython.core.display.Javascript object>

In [3]:
%%javascript

var data2 = [
  ["", "Maserati", "Mazda", "Mercedes", "Mini", "Mitsubishi"],
  ["2009b", 0, 2941, 4303, 354, 5814],
  ["2010", 5, 2905, 2867, 412, 5284],
  ["2011", 4, 2517, 4822, 552, 6127],
  ["2012", 2, 2422, 5399, 776, 4151]
];

element.handsontable({
  data: data2,
  minSpareCols: 1,
  minSpareRows: 1,
  rowHeaders: true,
  colHeaders: true,
  contextMenu: true
});


<IPython.core.display.Javascript object>

In [10]:
%%javascript
element.handsontable({
  data: data2,
  minSpareCols: 1,
  minSpareRows: 1,
  rowHeaders: true,
  colHeaders: true,
  contextMenu: true
});


<IPython.core.display.Javascript object>

In [ ]:
%%html

<p style="font-size: 20px"><strong>Handsontable</strong> is a minimalistic Excel-like <span class="nobreak">data grid</span>
  editor
  for HTML, JavaScript &amp; jQuery</p>

<div id="hs_example" class="handsontable"></div>

In [ ]:
%%javascript

require(['handsontable'], function (handsontable){
    console.log("in require->handsontable");
    
  var data = [
    ["", "Maserati", "Mazda", "Mercedes", "Mini", "Mitsubishi"],
    ["2009", 0, 2941, 4303, 354, 5814],
    ["2010", 5, 2905, 2867, 412, 5284],
    ["2011", 4, 2517, 4822, 552, 6127],
    ["2012", 2, 2422, 5399, 776, 4151]
  ];
  
  $('#hs_example').handsontable({
    data: data,
    minSpareRows: 1,
    colHeaders: true,
    contextMenu: true
  });
  
  
  function bindDumpButton() {
      $('body').on('click', 'button[name=dump]', function () {
        var dump = $(this).data('dump');
        var $container = $(dump);
        console.log('data of ' + dump, $container.handsontable('getData'));
      });
    }
  bindDumpButton();    
    
});

In [ ]:
display(hdf._widget)

In [ ]:
hdf = scisheets_widget.HandsonDataFrame(df)
hdf.show()

In [ ]:
display(hdf)

In [ ]:
data = np.random.randint(size=(3, 5), low=100, high=900)
df = pd.DataFrame(data)

In [ ]:
df = pd.DataFrame({'a':[1,3,5,7,4,5,6,4,7,8,9],
                   'b':[3,5,6,2,4,6,7,8,7,8,9]})

In [12]:
%%javascript
$element.handsontable({data:data1});

<IPython.core.display.Javascript object>

In [21]:
%%javascript
element.text('Hi')
console.log('There')

<IPython.core.display.Javascript object>

In [7]:
%%javascript

var table_id = 0;

this.$table = $('<div />')
                .attr('id', 'table_' + (table_id++))
                .appendTo(element);

this.$table.handsontable({data:data1});

// this.element.handsontable({data:data1});
// console.log('how?')

<IPython.core.display.Javascript object>

In [ ]:
%%javascript

var div = document.createElement('div');

div.innerHTML = '<span class="msg">Hello world.</span>';

In [ ]:
%%javascript
var output_area = this;
// find my cell element
var cell_element = output_area.element.parents('.cell');
// which cell is it?
var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
// get the cell object
var cell = Jupyter.notebook.get_cell(cell_idx);



## Test to make sure widgets work in notebook

In [22]:
import ipywidgets as widgets

In [23]:
widgets.IntSlider()

A Jupyter Widget